<a href="https://colab.research.google.com/github/carlibeisel/pod_pou_lulcc/blob/main/subset_LULCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modified by Carli Beisel

Adapted from code written by Bridget Bittmann (2023, Github: bridgetmarie24)

Date originally created: March 14, 2022

Date modified: April 16, 2024

Purpose: This script uses LCMAP data to calculate annual percent of each land class from 1987 to 2020 for each POU

In [ ]:
## IMPORT PACKAGES ##

import numpy as np #basic computation
#!pip install geopandas
import geopandas as gpd #geopandas for .shp
import matplotlib.pyplot as plt #to create plots
import pandas as pd #to create dataframes and export .csv
!pip install rasterio
import rasterio as rso #import GeoTiff files
from rasterio.mask import mask #to crop data to a boundary
from rasterio.plot import show #to plot the image
from rasterio.crs import CRS
from shapely.ops import unary_union #creates boundary of shapefile
import json #imports metadata
!pip install rioxarray #to clip rasters to a .shp file
import rioxarray as rxr
from rasterio.warp import calculate_default_transform, reproject, Resampling
!pip install pylandstats
import pylandstats #to perform landscape metrics
from pylandstats import landscape
from pylandstats import SpatioTemporalAnalysis #to calculate landscape metrics through time
import glob
import os
import matplotlib.lines as lines
import matplotlib.patches as patch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.2 MB/s eta 0:00:00


WARNING  Module /usr/local/lib/python3.10/dist-packages/pylandstats/landscape.py has not been compiled for         
         Transonic-Pythran

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
## ---------------------- ##
## SUBSET GEOSPATIAL DATA ##
## ---------------------- ##

#import rasterio
#rasterio.Env(GTIFF_SRS_SOURCE='EPSG')

shp_file = gpd.read_file('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/POUs/POUs_EDIT_060622_Merge.shp') #open shapefile
names = shp_file['WaterRight']
files = glob.glob('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmaps/*.tiff') #get all the years of cdl imagery
data =[]
for i in range(len(files)):
  data.append(rso.open(files[i])) #open cdl image and append to a list
  print(rso.open(files[i])) #added to check
shp = shp_file.to_crs(data[1].crs) #reproject the shp file to same projection
years = np.arange(1987, 2021) #years of LCMAP data
collection = []
for i in range(len(shp)):
  for n in range(len(years)):
    dataset = data[n]
    year_out = dataset.name[93:97] #change these numbers if things aren't working
    print(year_out) #added
    extent = gpd.GeoSeries(shp['geometry'][i]) #get the geometry from shapefile
    coords = [json.loads(extent.to_json())['features'][0]['geometry']] #gets coordinates for rasterio input
    out_img, out_transform = mask(dataset=data[n], shapes=coords, crop=True, nodata=0) #crop the data to the shapefile
    out_meta = data[n].meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_img.shape[1],
                     "width": out_img.shape[2],
                     "transform": out_transform})
    # Merge original file name with init_landcover to denote that it is the initial land cover data for Janus
    in_file = files[n]
    out_filename = os.path.join('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmap_masked/'+names[i]+'_'+year_out+'.tif') #create a file name to export to
    print(out_filename) #added
    # Save clipped land cover coverage THIS WILL OVERWRITE FILES
    out_tiff = rso.open(out_filename, 'w', **out_meta)
    out_tiff.write(np.squeeze(out_img, 0), 1)
    out_tiff.close()
    collection.append(out_img)

<open DatasetReader name='/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmaps/LCMAP_CU_1987_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmaps/LCMAP_CU_1986_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmaps/LCMAP_CU_1989_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmaps/LCMAP_CU_1988_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmaps/LCMAP_CU_1990_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmaps/LCMAP_CU_1991_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_fil

In [ ]:
## ---------------------------------------- ##
## Import multiple rasters into PyLandStats ##
## ---------------------------------------- ##
years = np.arange(1987,2021)
temporal_group = []
for i in range(len(names)):
  files= sorted(glob.glob('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/lcmap_masked/'+names[i]+'_*.tif')) #name for all the csv files
  sta = SpatioTemporalAnalysis(files, dates=years, nodata=0) #import all CDL rasters and mask
  temporal_group.append(sta)


In [ ]:
# ------------------------------- #
# CALCULATE THE CLASS PROPORTIONS #
# ------------------------------- #

proportions = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_class_metrics_df(temporal_group[i], metrics=['proportion_of_landscape'])
  df.to_csv('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/proportions/proportions2/'+names[i]+'_prop.csv')
  proportions.append(df)

In [ ]:
# ------------------------------- #
# CALCULATE CONFIGURATION METRICS #
# ------------------------------- #
#sticky code

config = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_landscape_metrics_df(temporal_group[i], metrics = ['contagion', 'largest_patch_index'])
  df.to_csv('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/configuration/'+names[i]+'_configuration.csv')
  config.append(df)

In [ ]:
# ----------------------------------------------------------------- #
# Put class proportions in the same format as configuration metrics #
# ----------------------------------------------------------------- #

# Import csv files into a list of dataframes

files = sorted(glob.glob('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/proportions/*_prop.csv'))
files_config = sorted(glob.glob('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/configuration/*_configuration.csv'))
names = list(sorted(shp_file['WaterRight']))

proportions = []
for i in files:
  data = pd.read_csv(i)
  proportions.append(data)
config = []
for i in files_config:
  data = pd.read_csv(i)
  config.append(data)

#Create new dataframes in same format as configuration metrics

new_df = []
for i in range(len(proportions)):
  df = pd.DataFrame(years, columns=['dates'])
  prop = proportions[i]
  df['DivName'] = names[i]
  df['class1_urban'] = prop['proportion_of_landscape'][prop['class_val'] == 1]
  df['class2_crops'] = prop['proportion_of_landscape'][prop['class_val'] == 2].values
  df = df.fillna(0)
  new_df.append(df)


In [ ]:
## ------------------------------------------ ##
## CALCULATE CHANGE IN URBAN AREA FOR MAPPING ##
## ------------------------------------------ ##

prop = pd.concat(new_df)

change = prop.groupby('DivName', as_index=False).class1_urban.agg(['min','max']).reset_index().fillna(0)
change['urb_change'] = change['max']-change['min']
change.to_csv('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/proportions/change.csv')

In [ ]:
## -------------------- ##
## MERGE TWO DATAFRAMES ##
## -------------------- ##

merged = []

for i in range(len(new_df)):
  df = new_df[i]
  con = config[i]
  df_merge = df.merge(con, on='dates', how='left')
  df_merge.to_csv('/content/gdrive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/lcmap_files/final_metrics/'+ names[i] +'.csv')
  merged.append(df_merge)